# Chapter 7

## 7.1   Handling Missing Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

The built-in Python `None` value is also treated as NA in object arrays:

In [4]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### Filtering Out Missing Data

In [5]:
from numpy import nan as NA

In [6]:
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [7]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, things are a bit more complex. You may want to drop rows or columns that are all NA or only those containing any NAs. `dropna` by default drops any row containing a missing value:

In [8]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


Passing `how='all'` will only drop rows that are all NA:

In [9]:
data.dropna(how = 'all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop columns in the same way, pass `axis=1`:

In [10]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [11]:
data.dropna(axis = 1, how = 'all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


A related way to filter out DataFrame rows tends to concern time series data. Suppose you want to keep only rows containing a certain number of observations. You can indicate this with the `thresh` argument:

In [12]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,1.125675,NaN,NaN
1,0.403963,NaN,NaN
2,1.222461,NaN,1.195817
3,1.547167,NaN,0.692215
4,0.624781,-0.815818,-0.455036
5,-0.106082,0.062730,0.521216
6,-1.250013,-0.537586,-1.291538


In [13]:
df.dropna()

,0,1,2
4,0.624781,-0.815818,-0.455036
5,-0.106082,0.062730,0.521216
6,-1.250013,-0.537586,-1.291538


In [14]:
df.dropna(thresh = 2)

,0,1,2
2,1.222461,NaN,1.195817
3,1.547167,NaN,0.692215
4,0.624781,-0.815818,-0.455036
5,-0.106082,0.062730,0.521216
6,-1.250013,-0.537586,-1.291538


### Filling In Missing Data

In [15]:
df

,0,1,2
0,1.125675,NaN,NaN
1,0.403963,NaN,NaN
2,1.222461,NaN,1.195817
3,1.547167,NaN,0.692215
4,0.624781,-0.815818,-0.455036
5,-0.106082,0.062730,0.521216
6,-1.250013,-0.537586,-1.291538


In [16]:
df.fillna(0)

,0,1,2
0,1.125675,0.000000,0.000000
1,0.403963,0.000000,0.000000
2,1.222461,0.000000,1.195817
3,1.547167,0.000000,0.692215
4,0.624781,-0.815818,-0.455036
5,-0.106082,0.062730,0.521216
6,-1.250013,-0.537586,-1.291538


Calling `fillna` with a dict, you can use a different fill value for each column:

In [17]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,1.125675,0.500000,0.000000
1,0.403963,0.500000,0.000000
2,1.222461,0.500000,1.195817
3,1.547167,0.500000,0.692215
4,0.624781,-0.815818,-0.455036
5,-0.106082,0.062730,0.521216
6,-1.250013,-0.537586,-1.291538


`fillna` returns a new object, but you can modify the existing object in-place:

In [18]:
_ = df.fillna(0, inplace = True)
df

,0,1,2
0,1.125675,0.000000,0.000000
1,0.403963,0.000000,0.000000
2,1.222461,0.000000,1.195817
3,1.547167,0.000000,0.692215
4,0.624781,-0.815818,-0.455036
5,-0.106082,0.062730,0.521216
6,-1.250013,-0.537586,-1.291538


The same interpolation methods available for reindexing can be used with `fillna`:

In [19]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,-1.068523,1.141210,0.710471
1,0.026547,-0.079523,1.060414
2,1.695127,NaN,-0.132634
3,1.112557,NaN,-0.337446
4,-1.109674,NaN,NaN
5,0.409249,NaN,NaN


In [20]:
df.fillna(method = 'ffill')

,0,1,2
0,-1.068523,1.141210,0.710471
1,0.026547,-0.079523,1.060414
2,1.695127,-0.079523,-0.132634
3,1.112557,-0.079523,-0.337446
4,-1.109674,-0.079523,-0.337446
5,0.409249,-0.079523,-0.337446


In [21]:
df.fillna(method = 'ffill', limit = 2)

,0,1,2
0,-1.068523,1.141210,0.710471
1,0.026547,-0.079523,1.060414
2,1.695127,-0.079523,-0.132634
3,1.112557,-0.079523,-0.337446
4,-1.109674,NaN,-0.337446
5,0.409249,NaN,-0.337446


With `fillna` you can do lots of other things with a little creativity. For example, you might pass the mean or median value of a Series:

In [22]:
data = pd.Series([1, NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## 7.2   Data Transformation

### Removing Duplicates

In [23]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'], 'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [24]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [25]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both of these methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates only based on the `'k1'` column:

In [26]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [27]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated` and `drop_duplicates` by default keep the first observed value combination. Passing `keep='last'` will return the last one:

In [28]:
data.drop_duplicates(['k1', 'k2'], keep = 'last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

In [29]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon', 'pastrami', 'honey ham', 'nova lox'], 
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [30]:
meat_to_animal = {'bacon': 'pig', 'pulled pork': 'pig', 'pastrami': 'cow', 'corned beef': 'cow', 'honey ham': 'pig', 
                  'nova lox': 'salmon'}
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [31]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [32]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

Using `map` is a convenient way to perform element-wise transformations and other data cleaning–related operations.

### Replacing Values

In [33]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [34]:
data.replace(-999., np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [35]:
data.replace([-999., -1000.], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [36]:
data.replace([-999., -1000.], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [37]:
data.replace({-999.: np.nan, -1000.: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes 

In [38]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)), index = ['Ohio', 'Colorado', 'New York'], columns = ['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [39]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [40]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [41]:
data.rename(index = str.title, columns = str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [42]:
data.rename(index = {'OHIO': 'INDIANA'}, columns = {'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [43]:
data.rename(index = {'OHIO': 'INDIANA'}, inplace = True)

In [44]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning

In [45]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special `Categorical` object. The output you see describes the bins computed by `pandas.cut`. You can treat it like an array of strings indicating the bin name; internally it contains a `categories` array specifying the distinct category names along with a labeling for the `ages` data in the `codes` attribute:

In [48]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [50]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [51]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

Consistent with mathematical notation for intervals, a parenthesis means that the side is *open*, while the square bracket means it is *closed* (inclusive). You can change which side is closed by passing `right=False`:

In [52]:
pd.cut(ages, [18, 26, 36, 61, 100], right = False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [55]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels = group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

If you pass an integer number of bins to `cut` instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data. Consider the case of some uniformly distributed data chopped into fourths:

In [56]:
data = np.random.rand(20)
pd.cut(data, 4, precision = 2)

[(0.75, 1.0], (0.5, 0.75], (0.0085, 0.26], (0.75, 1.0], (0.5, 0.75], ..., (0.75, 1.0], (0.75, 1.0], (0.5, 0.75], (0.5, 0.75], (0.75, 1.0]]
Length: 20
Categories (4, interval[float64, right]): [(0.0085, 0.26] < (0.26, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

The `precision=2` option limits the decimal precision to two digits.

A closely related function, `qcut`, bins the data based on sample quantiles. Depending on the distribution of the data, using `cut` will not usually result in each bin having the same number of data points. Since `qcut` uses sample quantiles instead, by definition you will obtain roughly equal-size bins:

In [58]:
data = np.random.randn(1000)
cats = pd.qcut(data, 4)
cats

[(0.0575, 0.71], (-0.667, 0.0575], (0.0575, 0.71], (-0.667, 0.0575], (-2.916, -0.667], ..., (-2.916, -0.667], (0.0575, 0.71], (0.0575, 0.71], (-2.916, -0.667], (-2.916, -0.667]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.916, -0.667] < (-0.667, 0.0575] < (0.0575, 0.71] < (0.71, 3.081]]

In [59]:
pd.value_counts(cats)

(-2.916, -0.667]    250
(-0.667, 0.0575]    250
(0.0575, 0.71]      250
(0.71, 3.081]       250
dtype: int64

Similar to `cut` you can pass your own quantiles (numbers between 0 and 1, inclusive):

In [61]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1])

[(0.0575, 1.326], (-1.254, 0.0575], (0.0575, 1.326], (-1.254, 0.0575], (-1.254, 0.0575], ..., (-1.254, 0.0575], (0.0575, 1.326], (0.0575, 1.326], (-2.916, -1.254], (-1.254, 0.0575]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.916, -1.254] < (-1.254, 0.0575] < (0.0575, 1.326] < (1.326, 3.081]]

### Detecting and Filtering Outliers

In [63]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.030118,0.037850,0.011269,0.032536
std,1.037930,1.002466,1.017869,0.981912
min,-3.445759,-2.933484,-3.571518,-2.877545
25%,-0.711041,-0.603016,-0.651866,-0.672543
50%,-0.037443,0.039030,0.014247,0.034042
75%,0.642440,0.676620,0.667214,0.698767
max,3.173891,2.921368,4.226766,3.462968


In [66]:
col = data[2]
col[np.abs(col) > 3]

336   -3.571518
353    4.226766
474   -3.077789
724   -3.021853
Name: 2, dtype: float64

To select all rows having a value exceeding 3 or –3, you can use the `any` method on a boolean DataFrame:

In [67]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,3.173891,-0.004864,0.257166,0.325821
336,-1.384587,-0.195743,-3.571518,-0.381886
342,0.920682,1.892429,-0.001528,3.060492
353,-0.844671,0.953225,4.226766,1.639019
427,3.038265,0.505477,0.015347,0.034414
474,-0.764960,-0.267429,-3.077789,-0.349382
704,-3.445759,-0.039336,-0.542813,0.127996
712,1.898702,-0.784383,0.530531,3.446125
724,0.688185,-0.633120,-3.021853,0.692362
857,-3.064742,1.317840,-0.122421,-2.184721


Values can be set based on these criteria. Here is code to cap values outside the interval –3 to 3:

In [68]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.029491,0.037850,0.010713,0.031567
std,1.034736,1.002466,1.011351,0.978761
min,-3.000000,-2.933484,-3.000000,-2.877545
25%,-0.711041,-0.603016,-0.651866,-0.672543
50%,-0.037443,0.039030,0.014247,0.034042
75%,0.642440,0.676620,0.667214,0.698767
max,3.000000,2.921368,3.000000,3.000000


In [69]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,1.0,-1.0
1,1.0,1.0,1.0,1.0
2,1.0,-1.0,1.0,-1.0
3,1.0,1.0,-1.0,1.0
4,1.0,-1.0,-1.0,-1.0


### Permutation and Random Sampling

Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do using the `numpy.random.permutation` function. Calling `permutation` with the length of the axis you want to permute produces an array of integers indicating the new ordering:

In [71]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [72]:
sampler = np.random.permutation(5)
sampler

array([2, 1, 0, 4, 3])

In [73]:
df.take(sampler)

,0,1,2,3
2,8,9,10,11
1,4,5,6,7
0,0,1,2,3
4,16,17,18,19
3,12,13,14,15


To select a random subset without replacement, you can use the `sample` method on Series and DataFrame:

In [74]:
df.sample(n = 3)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
3,12,13,14,15


To generate a sample *with* replacement (to allow repeat choices), pass `replace=True` to `sample`:

In [75]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n = 10, replace = True)
draws

1    7
4    4
3    6
2   -1
3    6
0    5
2   -1
4    4
4    4
0    5
dtype: int64

### Computing Indicator/Dummy Variables

In [76]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [78]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [79]:
dummies = pd.get_dummies(df['key'], prefix = 'key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [82]:
df['data1']

0    0
1    1
2    2
3    3
4    4
5    5
Name: data1, dtype: int64

In [83]:
df[['data1']]

,data1
0,0
1,1
2,2
3,3
4,4
5,5


A useful recipe for statistical applications is to combine `get_dummies` with a discretization function like `cut`:

In [84]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [85]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## 7.3   String Manipulation

### Regular Expressions

In [87]:
import re
text = """Dave dave@google.com
    Steve steve@gmail.com
    Rob rob@gmail.com
    Ryan ryan@yahoo.com
    """
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
regex = re.compile(pattern, flags = re.IGNORECASE)

In [88]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

`search` returns a special match object for the first email address in the text.

In [89]:
m = regex.search(text)
text[m.start():m.end()]

'dave@google.com'

`regex.match` returns `None`, as it only will match if the pattern occurs at the start of the string:

In [91]:
print(regex.match(text))

None


Suppose you wanted to find email addresses and simultaneously segment each address into its three components: username, domain name, and domain suffix. To do this, put parentheses around the parts of the pattern to segment:

In [92]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags = re.IGNORECASE)

In [93]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

`findall` returns a list of tuples when the pattern has groups:

In [95]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

`sub` also has access to groups in each match using special symbols like `\1` and `\2`. The symbol `\1` corresponds to the first matched group, `\2` corresponds to the second, and so forth:

In [96]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
    Steve Username: steve, Domain: gmail, Suffix: com
    Rob Username: rob, Domain: gmail, Suffix: com
    Ryan Username: ryan, Domain: yahoo, Suffix: com
    


### Vectorized String Functions in pandas

In [97]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com', 'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [98]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [99]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [101]:
data.str.findall(pattern, flags = re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [103]:
matches = data.str.match(pattern, flags = re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [105]:
data.str.get(1)

Dave       a
Steve      t
Rob        o
Wes      NaN
dtype: object

In [106]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object